In [6]:
import torch

In [7]:
import torch.nn as nn

In [60]:
class Decoder(nn.Module):
    def __init__(self, embedding_size, vocab_size, hidden_size):
        super(Decoder,self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size,embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.fc = nn.Linear(hidden_size,vocab_size)
        
    def forward(self, features, captions):
        time_steps = captions.size(1)
        batch_size = features.size(0)
        caption_embeddings = self.embedding(captions)
        hx = torch.zeros(batch_size, self.hidden_size)  
        cx = torch.zeros(batch_size, self.hidden_size)
        output = []
        for i in range(time_steps):
            if i == 0:
                hx, cx = self.lstm(features, (hx,cx))
            else:
                hx, cx = self.lstm(caption_embeddings[:,i-1,:], (hx, cx))
            output.append(hx)
            
        raw_scores = self.fc(torch.stack(output,1))
        
        return raw_scores
        
        